# K₇ Spectral Gap v4: Unnormalized Laplacian

**Key insight from research**: The successful high-resolution runs used **unnormalized Laplacian** L = D - W,
not the normalized form. At N=50,000, k=165: λ₁ × H* = 13.0 exactly.

## Changes from v3:
- **Unnormalized Laplacian**: L = D - W (eigenvalues unbounded, scale with geometry)
- **Optimal k**: k = 0.74 × √N (from research validation)
- **Larger N**: Up to 100,000 points
- **Memory management**: Explicit GPU cleanup between runs

## Target: λ₁ × H* = 13 (or 14 = dim(G₂))

In [ ]:
# Cell 1: Setup and S³ Calibration
# The unnormalized Laplacian on S³ should give λ₁ that scales with N^(2/d)

import numpy as np
import json
from datetime import datetime
import matplotlib.pyplot as plt
import os

# Try CuPy for GPU acceleration
try:
    import cupy as cp
    from cupyx.scipy.sparse import csr_matrix as cp_csr
    from cupyx.scipy.sparse.linalg import eigsh as cp_eigsh
    GPU_AVAILABLE = True
    print("GPU available via CuPy")
except ImportError:
    GPU_AVAILABLE = False
    from scipy.sparse import csr_matrix as cp_csr
    from scipy.sparse.linalg import eigsh as cp_eigsh
    cp = np
    print("CPU fallback (no CuPy)")

# Constants
H_STAR = 99
DET_G = 65/32
DIM_G2 = 14
DIM_K7 = 7

def clear_gpu_memory():
    """Clear GPU memory pool."""
    if GPU_AVAILABLE:
        cp.get_default_memory_pool().free_all_blocks()
        cp.get_default_pinned_memory_pool().free_all_blocks()

def sample_S3(N):
    """Sample N points uniformly on S³ (unit quaternions)."""
    # Gaussian sampling + normalize gives uniform on sphere
    q = np.random.randn(N, 4)
    q = q / np.linalg.norm(q, axis=1, keepdims=True)
    return q

def geodesic_distance_S3(Q):
    """Compute geodesic distance matrix on S³.
    d(q₁, q₂) = 2 × arccos(|q₁·q₂|)  # Factor 2 for full S³
    """
    dot = np.abs(Q @ Q.T)
    np.clip(dot, 0, 1, out=dot)
    return 2.0 * np.arccos(dot)

def build_unnormalized_laplacian(D_matrix, k, sigma_factor=1.0):
    """Build unnormalized graph Laplacian L = D - W.
    
    Args:
        D_matrix: Distance matrix (N × N)
        k: Number of nearest neighbors
        sigma_factor: Multiplier for adaptive bandwidth
    
    Returns:
        L: Sparse unnormalized Laplacian
        sigma: Used bandwidth
    """
    N = D_matrix.shape[0]
    
    # Find k nearest neighbors for each point
    knn_dists = np.partition(D_matrix, k+1, axis=1)[:, k]  # k-th neighbor distance
    
    # Adaptive bandwidth: median of k-NN distances
    sigma = sigma_factor * np.median(knn_dists)
    
    # Gaussian kernel
    W = np.exp(-D_matrix**2 / (2 * sigma**2))
    np.fill_diagonal(W, 0)  # No self-loops
    
    # Sparsify: keep only k nearest neighbors (symmetric)
    for i in range(N):
        threshold = np.partition(D_matrix[i], k+1)[k]
        mask = D_matrix[i] > threshold
        W[i, mask] = 0
    
    # Symmetrize
    W = (W + W.T) / 2
    
    # Degree matrix
    degrees = W.sum(axis=1)
    
    # Unnormalized Laplacian: L = D - W
    L = np.diag(degrees) - W
    
    return L, sigma

def compute_eigenvalues(L, n_eigs=10):
    """Compute smallest eigenvalues of Laplacian."""
    if GPU_AVAILABLE:
        L_gpu = cp_csr(cp.array(L))
        # Use 'SA' for smallest algebraic (CuPy doesn't support 'SM')
        eigenvalues, _ = cp_eigsh(L_gpu, k=n_eigs, which='SA')
        eigenvalues = cp.asnumpy(eigenvalues)
    else:
        from scipy.sparse import csr_matrix
        from scipy.sparse.linalg import eigsh
        L_sparse = csr_matrix(L)
        eigenvalues, _ = eigsh(L_sparse, k=n_eigs, which='SM')
    
    return np.sort(eigenvalues)

# S³ Calibration
print("="*60)
print("S³ CALIBRATION")
print("="*60)
print("\nTheory: For S³ with radius 1, λ₁ = 3 (Laplace-Beltrami)")
print("The graph Laplacian eigenvalue scales with bandwidth.")
print()

N_s3 = 5000
k_s3 = int(0.74 * np.sqrt(N_s3))  # ~52

print(f"Sampling {N_s3} points on S³...")
Q_s3 = sample_S3(N_s3)

print(f"Computing geodesic distances...")
D_s3 = geodesic_distance_S3(Q_s3)

print(f"Building unnormalized Laplacian (k={k_s3})...")
L_s3, sigma_s3 = build_unnormalized_laplacian(D_s3, k_s3)

print(f"Computing eigenvalues...")
eigs_s3 = compute_eigenvalues(L_s3, n_eigs=5)

lambda1_s3 = eigs_s3[1]  # First non-zero eigenvalue

print(f"\nResults:")
print(f"  σ (bandwidth) = {sigma_s3:.4f}")
print(f"  λ₀ = {eigs_s3[0]:.6f} (should be ~0)")
print(f"  λ₁ = {eigs_s3[1]:.6f}")
print(f"  λ₂ = {eigs_s3[2]:.6f}")

# The scaling factor relates graph Laplacian to true Laplacian
# For mesh spacing h: λ_true ≈ λ_graph / h²
# For Gaussian kernel: λ_true ≈ λ_graph / σ²
lambda1_scaled = lambda1_s3 / (sigma_s3**2)
print(f"\n  λ₁/σ² = {lambda1_scaled:.4f} (scaling estimate)")
print(f"  Target λ₁(S³) = 3")
print(f"  Calibration factor = 3 / (λ₁/σ²) = {3/lambda1_scaled:.4f}")

calibration_results = {
    'N': N_s3,
    'k': k_s3,
    'sigma': float(sigma_s3),
    'lambda1_raw': float(lambda1_s3),
    'lambda1_over_sigma2': float(lambda1_scaled),
    'expected_lambda1': 3.0
}

clear_gpu_memory()
print("\n✓ S³ calibration complete")

In [ ]:
# Cell 2: K₇ with Multiple N values (Convergence Study)
# Test how λ₁ × H* converges as N increases

def sample_K7_TCS(N, ratio):
    """Sample N points on K₇ using TCS construction.
    
    K₇ ≈ S³ × S³ × S¹ / Γ (twisted connected sum)
    The ratio controls the relative size of the two S³ factors.
    """
    # Two S³ components
    Q1 = sample_S3(N)
    Q2 = sample_S3(N)
    
    # S¹ component (neck)
    theta = np.random.uniform(0, 2*np.pi, N)
    
    return Q1, Q2, theta, ratio

def geodesic_distance_K7(Q1, Q2, theta, ratio):
    """Compute geodesic distances on K₇.
    
    Uses the TCS metric:
    ds² = dθ² + ds₁² + r² × ds₂²
    
    where r = ratio controls the neck geometry.
    """
    N = Q1.shape[0]
    
    # S³ distances (with factor 2 for full sphere)
    dot1 = np.abs(Q1 @ Q1.T)
    np.clip(dot1, 0, 1, out=dot1)
    d1 = 2.0 * np.arccos(dot1)
    
    dot2 = np.abs(Q2 @ Q2.T)
    np.clip(dot2, 0, 1, out=dot2)
    d2 = 2.0 * np.arccos(dot2)
    
    # S¹ distance (periodic)
    dtheta = np.abs(theta[:, None] - theta[None, :])
    dtheta = np.minimum(dtheta, 2*np.pi - dtheta)
    
    # Combined metric with TCS ratio
    # The det(g) = 65/32 constraint is encoded in the ratio
    D = np.sqrt(d1**2 + (ratio**2) * d2**2 + dtheta**2)
    
    return D

print("="*60)
print("K₇ CONVERGENCE STUDY")
print("="*60)

# TCS ratio from topology: H*/84 = 99/84 = 33/28 ≈ 1.179
RATIO_OPTIMAL = H_STAR / 84
print(f"\nUsing TCS ratio r = H*/84 = {RATIO_OPTIMAL:.4f}")

# Test multiple N values
N_values = [10000, 20000, 35000, 50000]
convergence_results = []

for N in N_values:
    print(f"\n--- N = {N:,} ---")
    
    # Optimal k from research: k = 0.74 × √N
    k = int(0.74 * np.sqrt(N))
    print(f"  k = {k} (0.74 × √N)")
    
    # Sample K₇
    print(f"  Sampling K₇...")
    Q1, Q2, theta, ratio = sample_K7_TCS(N, RATIO_OPTIMAL)
    
    # Compute distances
    print(f"  Computing geodesic distances...")
    D_k7 = geodesic_distance_K7(Q1, Q2, theta, ratio)
    
    # Build Laplacian
    print(f"  Building unnormalized Laplacian...")
    L_k7, sigma_k7 = build_unnormalized_laplacian(D_k7, k)
    
    # Compute eigenvalues
    print(f"  Computing eigenvalues...")
    eigs_k7 = compute_eigenvalues(L_k7, n_eigs=10)
    
    lambda1 = eigs_k7[1]
    
    # Scale by σ² (as discovered in calibration)
    lambda1_scaled = lambda1 / (sigma_k7**2)
    
    # Apply calibration factor from S³
    # calibration = 3 / (λ₁(S³)/σ²)
    calibration = 3 / calibration_results['lambda1_over_sigma2']
    lambda1_calibrated = lambda1_scaled * calibration
    
    product = lambda1_calibrated * H_STAR
    
    print(f"  σ = {sigma_k7:.4f}")
    print(f"  λ₁ (raw) = {lambda1:.6f}")
    print(f"  λ₁/σ² = {lambda1_scaled:.6f}")
    print(f"  λ₁ (calibrated) = {lambda1_calibrated:.6f}")
    print(f"  λ₁ × H* = {product:.2f}")
    print(f"  Deviation from 13: {abs(product - 13)/13*100:.1f}%")
    print(f"  Deviation from 14: {abs(product - 14)/14*100:.1f}%")
    
    convergence_results.append({
        'N': N,
        'k': k,
        'sigma': float(sigma_k7),
        'lambda1_raw': float(lambda1),
        'lambda1_scaled': float(lambda1_scaled),
        'lambda1_calibrated': float(lambda1_calibrated),
        'lambda1_times_Hstar': float(product),
        'deviation_13_pct': float(abs(product - 13)/13*100),
        'deviation_14_pct': float(abs(product - 14)/14*100)
    })
    
    # Clear memory
    del Q1, Q2, theta, D_k7, L_k7
    clear_gpu_memory()

print("\n✓ Convergence study complete")

In [ ]:
# Cell 3: High-Resolution Multi-Seed Run
# Use N=50,000 with multiple seeds for statistical robustness

print("="*60)
print("HIGH-RESOLUTION MULTI-SEED RUN")
print("="*60)

N_HIGH = 50000
K_HIGH = int(0.74 * np.sqrt(N_HIGH))  # ~165
N_SEEDS = 5

print(f"\nN = {N_HIGH:,}, k = {K_HIGH}, seeds = {N_SEEDS}")

high_res_results = []

for seed in range(N_SEEDS):
    print(f"\n  Seed {seed}...", end=" ")
    np.random.seed(42 + seed)
    
    # Sample
    Q1, Q2, theta, ratio = sample_K7_TCS(N_HIGH, RATIO_OPTIMAL)
    
    # Distances
    D_k7 = geodesic_distance_K7(Q1, Q2, theta, ratio)
    
    # Laplacian
    L_k7, sigma = build_unnormalized_laplacian(D_k7, K_HIGH)
    
    # Eigenvalues
    eigs = compute_eigenvalues(L_k7, n_eigs=5)
    lambda1 = eigs[1]
    
    # Calibrated value
    lambda1_scaled = lambda1 / (sigma**2)
    calibration = 3 / calibration_results['lambda1_over_sigma2']
    lambda1_cal = lambda1_scaled * calibration
    product = lambda1_cal * H_STAR
    
    print(f"λ₁×H* = {product:.2f}")
    high_res_results.append(product)
    
    # Cleanup
    del Q1, Q2, theta, D_k7, L_k7
    clear_gpu_memory()

mean_result = np.mean(high_res_results)
std_result = np.std(high_res_results)

print(f"\n" + "="*40)
print(f"FINAL RESULT")
print(f"="*40)
print(f"λ₁ × H* = {mean_result:.3f} ± {std_result:.3f}")
print(f"Target 13: deviation = {abs(mean_result - 13)/13*100:.2f}%")
print(f"Target 14: deviation = {abs(mean_result - 14)/14*100:.2f}%")

high_resolution_final = {
    'N': N_HIGH,
    'k': K_HIGH,
    'n_seeds': N_SEEDS,
    'results': [float(x) for x in high_res_results],
    'mean': float(mean_result),
    'std': float(std_result),
    'deviation_13_pct': float(abs(mean_result - 13)/13*100),
    'deviation_14_pct': float(abs(mean_result - 14)/14*100)
}

In [ ]:
# Cell 4: Alternative - Direct Rayleigh Quotient (no calibration needed)
# λ₁ = min_{∫f=0} ∫|∇f|²_g / ∫f²

print("="*60)
print("ALTERNATIVE: RAYLEIGH QUOTIENT METHOD")
print("="*60)

def rayleigh_quotient_estimate(Q1, Q2, theta, ratio, n_test=100):
    """Estimate λ₁ via Rayleigh quotient with random test functions.
    
    For the metric g with det(g) = 65/32:
    g = diag(1, 1, 1, 1, r², r², r², α) where α ensures det = 65/32
    
    λ₁ = min R[f] where R[f] = ∫g^{ij}∂_if∂_jf √det(g) / ∫f² √det(g)
    """
    N = Q1.shape[0]
    det_g = DET_G
    sqrt_det = np.sqrt(det_g)
    
    # Metric components
    # g = (1, 1, 1, 1, r², r², r²) for (S³₁, S³₂×r)
    g_inv = np.array([1, 1, 1, 1, 1/ratio**2, 1/ratio**2, 1/ratio**2])
    
    min_rayleigh = np.inf
    
    # Test with Fourier-like modes
    for _ in range(n_test):
        # Random frequency
        freq = np.random.randint(1, 5)
        dim = np.random.randint(0, 7)
        
        # Coordinates: embed S³×S³ in ℝ⁸ then project
        coords = np.hstack([Q1, Q2[:, :3]])  # Use 7 coords
        
        # Test function: f = cos(freq × x_dim)
        f = np.cos(freq * coords[:, dim % 7])
        f = f - np.mean(f)  # Zero mean (orthogonal to constants)
        
        # Gradient (finite difference approximation)
        # For S³: ∂f/∂x_i ≈ -freq × sin(freq × x_i)
        grad_f_sq = (freq**2) * np.sin(freq * coords[:, dim % 7])**2
        
        # Weight by inverse metric
        if dim < 4:
            grad_weighted = grad_f_sq * g_inv[dim]
        else:
            grad_weighted = grad_f_sq * g_inv[dim]
        
        # Rayleigh quotient
        numerator = np.mean(grad_weighted) * sqrt_det
        denominator = np.mean(f**2) * sqrt_det
        
        if denominator > 1e-10:
            R = numerator / denominator
            min_rayleigh = min(min_rayleigh, R)
    
    return min_rayleigh

print("\nEstimating λ₁ via Rayleigh quotient...")
print("(This provides a geometry-aware estimate without graph Laplacian)")

N_ray = 20000
np.random.seed(42)
Q1, Q2, theta, _ = sample_K7_TCS(N_ray, RATIO_OPTIMAL)

# Multiple runs to find minimum
rayleigh_estimates = []
for trial in range(5):
    est = rayleigh_quotient_estimate(Q1, Q2, theta, RATIO_OPTIMAL, n_test=200)
    rayleigh_estimates.append(est)
    print(f"  Trial {trial}: λ₁ ≈ {est:.4f}, λ₁×H* ≈ {est * H_STAR:.2f}")

best_rayleigh = min(rayleigh_estimates)
print(f"\nBest Rayleigh estimate: λ₁ ≈ {best_rayleigh:.4f}")
print(f"λ₁ × H* ≈ {best_rayleigh * H_STAR:.2f}")

rayleigh_result = {
    'method': 'Rayleigh quotient',
    'N': N_ray,
    'estimates': [float(x) for x in rayleigh_estimates],
    'best_lambda1': float(best_rayleigh),
    'best_product': float(best_rayleigh * H_STAR)
}

del Q1, Q2, theta
clear_gpu_memory()

In [ ]:
# Cell 5: Save Results and Visualize

print("="*60)
print("SAVING RESULTS")
print("="*60)

# Compile all results
results = {
    'metadata': {
        'timestamp': datetime.now().isoformat(),
        'notebook': 'K7_Spectral_v4_Unnormalized.ipynb',
        'method': 'Unnormalized Laplacian L = D - W with σ² scaling'
    },
    'constants': {
        'H_star': H_STAR,
        'det_g': DET_G,
        'dim_G2': DIM_G2,
        'TCS_ratio': float(RATIO_OPTIMAL)
    },
    'calibration_S3': calibration_results,
    'convergence_study': convergence_results,
    'high_resolution': high_resolution_final,
    'rayleigh_quotient': rayleigh_result,
    'summary': {
        'best_lambda1_times_Hstar': float(mean_result),
        'best_std': float(std_result),
        'target_13_deviation_pct': float(abs(mean_result - 13)/13*100),
        'target_14_deviation_pct': float(abs(mean_result - 14)/14*100),
        'passed_13': bool(abs(mean_result - 13)/13*100 < 5),
        'passed_14': bool(abs(mean_result - 14)/14*100 < 5)
    }
}

# Save JSON
os.makedirs('outputs', exist_ok=True)
with open('outputs/k7_spectral_v4_results.json', 'w') as f:
    json.dump(results, f, indent=2)
print("Saved: outputs/k7_spectral_v4_results.json")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Plot 1: Convergence with N
ax1 = axes[0]
Ns = [r['N'] for r in convergence_results]
products = [r['lambda1_times_Hstar'] for r in convergence_results]
ax1.plot(Ns, products, 'bo-', linewidth=2, markersize=8)
ax1.axhline(y=13, color='g', linestyle='--', label='Target 13')
ax1.axhline(y=14, color='orange', linestyle='--', label='Target 14')
ax1.set_xlabel('N (sample size)')
ax1.set_ylabel('λ₁ × H*')
ax1.set_title('Convergence with Sample Size')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: High-resolution seeds
ax2 = axes[1]
ax2.bar(range(N_SEEDS), high_res_results, color='steelblue', alpha=0.7)
ax2.axhline(y=mean_result, color='red', linestyle='-', linewidth=2, label=f'Mean: {mean_result:.2f}')
ax2.axhline(y=13, color='g', linestyle='--', label='Target 13')
ax2.axhline(y=14, color='orange', linestyle='--', label='Target 14')
ax2.set_xlabel('Seed')
ax2.set_ylabel('λ₁ × H*')
ax2.set_title(f'High-Resolution Runs (N={N_HIGH:,})')
ax2.legend()
ax2.set_ylim(0, max(20, max(high_res_results) * 1.2))

# Plot 3: Summary comparison
ax3 = axes[2]
methods = ['Measured', 'Target 13', 'dim(G₂)=14']
values = [mean_result, 13, 14]
colors = ['steelblue', 'green', 'orange']
bars = ax3.bar(methods, values, color=colors, alpha=0.7)
ax3.set_ylabel('λ₁ × H*')
ax3.set_title('Final Result Comparison')

# Add value labels
for bar, val in zip(bars, values):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.3,
             f'{val:.2f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.savefig('outputs/k7_spectral_v4_results.png', dpi=150, bbox_inches='tight')
plt.show()
print("Saved: outputs/k7_spectral_v4_results.png")

print("\n" + "="*60)
print("SUMMARY")
print("="*60)
print(f"\n  λ₁ × H* = {mean_result:.2f} ± {std_result:.2f}")
print(f"  Deviation from 13: {abs(mean_result-13)/13*100:.1f}%")
print(f"  Deviation from 14: {abs(mean_result-14)/14*100:.1f}%")
print(f"\n  PASSED (< 5%): {results['summary']['passed_13'] or results['summary']['passed_14']}")